# Dealing with multiple Radial Velocity measurements

It is a common task in astronomy to calculate a final measured quantity (and error) when there are multiple individual measurements for a particular object. This is often the case when dealing with radial velocities (RVs) for stars, particularly in the era of large spectroscopic surveys such as RAVE, Gaia DR2 (soon to be DR3) and GALAH.

We also know that approximately half of all stars belong in multiple systems, and depending on their orbital properties (e.g., physical separation, eccentricity, inclination), their multiplicity status is given away by the fact that the components vary in RV from as low as 1 to 100 km/s. Whilst obvious temporal changes in RV are a giveaway for identifying multiples, it should be noted there is always some (usually small) probability that multiple systems evade detection because they are observed at the same orbital phase!

### With this in mind, the following program:

- reads in a table of RV measurements for any set of stars

- makes a decision on whether the star is multiple or likely single
- calculates a final RV from the individual measurements.

### Some points before we begin:

1. The table consists of a star identifier (that must be exactly the same if multiple entries are present), a reference to the source catalogue, the RV, and the RV error.

2. For the cases where no RV error is calculated (tut, tut), for lack of a better method, we have assumed this error to be twice the median error bar from the whole sample of RV measurements. It is the user's discretion whether or not to include data that are missing errors (but sometimes even these measurements have intrinsic value if, say, there are no other measurements).

3. The confidence placed in the reliability of each RV measurement is equal regardless of provenance.

4. In the rare case where two RV measurements for a source differ by an amount greater than the physical maximum for an equal mass binary system, $\delta RV \geq 437\times\frac{\sqrt{M/M_{\odot}}}{\sqrt{a/R_{\odot}}}$ km/s -- *Bowers & Deeming 1984, 1984astr.book.....B*) then we discard the largest outlier.

### Part 1: Reading in the catalogue and pre-cleaning

Import python modules

In [ ]:
from astropy.table import unique, Table
from astropy.io import ascii
from astropy.stats import sigma_clip
import numpy as np
import itertools

Read in the input file
The minimum requirement is;
- name
- RV
- eRV
- reference

Currently the following line would read in a basic ascii (space-separated)
file, but "ascii.read" can take all kinds of formats, which are described
in: https://docs.astropy.org/en/stable/io/ascii/read.html


In [ ]:
f_in = ascii.read("AllRVs.dat", format='basic', delimiter=' ', guess=False)

In case of missing errors, use 2x the median error from all observations. In our example missing errors are marked as "-999". Please modify this line to make sense of your own dataset.

In [ ]:
f_in["eRV"] = np.where(f_in["eRV"] < -99, 2.0*np.median(f_in["eRV"]), f_in["eRV"])

Get a list of unique names for each star

In [ ]:
f_un = unique(f_in, keys='name')

The maximum possible RV difference between two observations was given earlier (point 4). One may wish to include columns in their input table of estimated mass and radius. In our case we are dealing mainly with  early K-type pre-main sequence stars: mass = $0.8M_{\odot}$ and radius = $0.7R_{\odot}$.


In [ ]:
mass, radius = 0.8, 0.7
d_max = 437.*np.sqrt(mass)/np.sqrt(radius)

Now we want to remove any entries in the catalogue that cause $\delta RV \geq 437\times\frac{\sqrt{M/M_{\odot}}}{\sqrt{a/R_{\odot}}}$ km/s. Note that we only remove the object that we believe is causing the (unphysically) large RV difference. To do so we loop over each unique star name, and if there are $\geq 2$ measurements, construct an array of pairwise RV differences, for example:
- 3 measurements: A vs B, A vs C, B vs C
- 4 measurements: A vs B, A vs C, A vs D, B vs C, B vs D, C vs D

The arrays are sorted from largest to smallest values in $\Delta RV$. 

In [ ]:
for f in f_un:
    g = f_in[f_in["name"] == f["name"]]
#if there are 2 or more measurements we want to test how consistent the RVs are.
    x = len(g)
# Remove stars that cause RV differences larger than d_max.
# Get pair-wise RV measurements and find their differences
# using "itertools.combinations"
    if x >= 2:
        p = list(itertools.combinations(g["RV"], 2))
        b = list((i,j) for ((i,_),(j,_)) in itertools.combinations(enumerate(g["RV"]), 2))
        c1, c0, darr = [g["eRV"][c[1]] for c in b], [g["eRV"][c[0]] for c in b], [abs(z[1]-z[0]) for z in p]
# for the new pairwise arrays, find the locations where the RV difference > d_max
        arr_bad = np.array(b)[np.array(darr) > d_max]
# if there are any examples where the RV difference > d_max
   # and at least 2 cases where this happens
        if np.any(arr_bad) and len(arr_bad) >= 2:
            ind_outliers = np.array(arr_bad).ravel()
   # find the entry which is causing the big RV difference (occurs the most often)
            g_r = g[np.bincount(ind_outliers).argmax()]
            f_in_r = [(f_in["cat"] == g_r[1]) & (f_in["name"] == g_r[0])]
            f_in_ind = np.where(f_in_r)[1]
   # remove this entry...
            f_in.remove_row(f_in_ind[0])

   # if only one occasion where RV difference is large (i.e., 2 measurements)
        if np.any(arr_bad) and len(arr_bad) < 2:
            ind_outliers = np.array(arr_bad).ravel()
            g_r = g[ind_outliers]
   # select the entry with the largest RV error as the outlier to remove
            g_max = g[g_r["eRV"] == max(g_r["eRV"])]
            f_in_r = [(f_in["cat"] == g_max["cat"]) & (f_in["name"] == g_max["name"])]
            f_in_ind = np.where(f_in_r)[1]
            f_in.remove_row(f_in_ind[0])

### Part 2: The decision process for multiple/single stars

The next job is to combine multiple RV measurements again (this time without any extreme RV outliers), and create an array of pairwise RV differences. Sort the array in descending values of RV difference. If the RV difference is larger than the following criteria:

$\Delta RV > \zeta\times max(\sigma_{RV_{1}}, \sigma_{RV_{2}})$, where $\zeta = 3.0$ (but can be altered).

Then we flag the star as a likely binary. If this criteria is not matched, move to the next lowest RV difference and recalculate. If, at the end this still isn't matched, then flag the star as a probable single.

In [ ]:
Nsig = 3.0

Create an empty array "F_arr", which will store the designated multiplicity status, where:
- F_arr = 5: multiple, at least one $\Delta RV > \zeta\times max(\sigma_{RV_{1}}, \sigma_{RV_{2}})$
- F_arr = 3: (probably) single, every $\Delta RV \leq \zeta\times max(\sigma_{RV_{1}}, \sigma_{RV_{2}})$
- F_arr = 1: = only 1 RV measurement, we can't say anything about multiplicity status
- F_arr = 0: = no RV measurements available


In [ ]:
f_un = unique(f_in, keys='name')
F_Arr = []

for f in f_un:
    g = f_in[f_in["name"] == f["name"]]
#if there are 2 or more measurements we want to test how consistent the RVs are.
    x = len(g)

    if x >= 2:
        p = list(itertools.combinations(g["RV"], 2))
        b = list((i,j) for ((i,_),(j,_)) in itertools.combinations(enumerate(g["RV"]), 2))
        c1, c0, darr = [g["eRV"][c[1]] for c in b], [g["eRV"][c[0]] for c in b], [abs(z[1]-z[0]) for z in p]
        z = np.argsort(darr)
        RVd_arr = [[darr[z[i]], c1[z[i]], c0[z[i]]] for i in z]# if darr[i] < d_max]
            
# fill out the RV flag arrays based on the RV differences (and errors)
# 5 = multiple, dRV > d_max
# 3 = single, dRV <= d_max
# 1 = only 1 RV
# 0 = no RVs
        for j in range(len(RVd_arr)):
            eRV_max = max(RVd_arr[j][2], RVd_arr[j][1])
            if RVd_arr == None:
                F_Arr.append(0)
                break
            if len(z) > 1:
# if at any point |RV| > N*max(eRV) then we have no choice
# but to flag it as a probable multiple.
                if (RVd_arr[j][0] > Nsig*eRV_max):
                    F_Arr.append(5)
                    break
# |RV| <= N*max(eRV)
# could be single, check the next RV difference
                if (RVd_arr[j][0] <= Nsig*eRV_max):
                    j=j+1
# if after all the iterations we get here and we still didn't see any
# evidence of binarity, flag it as a single star.
                    if j == len(z)-1:
                         F_Arr.append(3)
                         break
            if len(z) == 1:
                if (RVd_arr[j][0] > Nsig*eRV_max):
                    F_Arr.append(5)
#                    break
                if (RVd_arr[j][0] <= Nsig*eRV_max):
                    F_Arr.append(3)
#                    break

    if x == 1:
        if g["RV"][0] > -900:
            F_Arr.append(1)
        else:
            F_Arr.append(0)
    if x == 0:
        F_Arr.append(0)

### Part 3: obtaining a final RV measurement
Regardless of the multiplicity status assigned in Part 2, we now calculate a final RV measurement, and flag "how" we measure RV using a measurement flag (Mflag).

#### If the star is a probable single, there are two paths to calculating a final RV.

1. If the standard deviation in the error bars is less than the standard deviation in the measurements, then take the inverse weighted mean, using the square of the error bars as weights. Mflag = 2.
2. Otherwise, adopt the final RV as the value corresponding to the lowest error bar. Mflag = 3.

The associtated error in the weighted mean is found using equation (2) and (3) at this link: http://seismo.berkeley.edu/~kirchner/Toolkits/Toolkit_12.pdf

#### If the star is likely to be a multiple, use path 1 described above.
Mflag = 4.

The cases with only 1 or 0 RV measurements is trivial, Mflag = 1 and 0, respectively.

Two error bars are provided:
- "sRV", the error in the weighted mean.
- "eRV", the average value of the individual error bars.

In [ ]:
RV_f, eRV_f, sRV_f, RV_T = [], [], [], []

for k, l in enumerate(f_un):
    g = f_in[f_in["name"] == l["name"]]
    sdRV  = np.std(g["RV"])
    sdeRV = np.std(g["eRV"])
#if there are 2 or more measurements we want to test how consistent the RVs are.
    x = len(g)
    if x >= 2:
# if we don't class the star as a possible binary
        w = np.array(1.0/(g["eRV"]**2))
        if F_Arr[k] != 5:
# if the std in the errors is less than the std in the mean RV
# take the weighted average and error

# weighted error is found like this:
# http://seismo.berkeley.edu/~kirchner/Toolkits/Toolkit_12.pdf

            if sdeRV <= sdRV:
                w_RV = np.average(g["RV"], weights=w)
                val = np.array(g["RV"]**2)
                var = ((np.sum(w*val)/np.sum(w))-w_RV**2) * (x/(x-1))
                RV_f.append(w_RV)
                sRV_f.append(np.sqrt(var/x))
                eRV_f.append(np.average(g["eRV"]))
                RV_T.append(2)
            else:
                RV_f.append(g["RV"][np.argsort(g["eRV"][0])][0])
                eRV_f.append(g["eRV"][np.argsort(g["eRV"][0])][0])
                sRV_f.append(np.average(g["eRV"]))
                RV_T.append(3)
        if F_Arr[k] == 5:
            w_RV = np.average(g["RV"], weights=w)
            val = np.array(g["RV"]**2)
            var = ((np.sum(w*val)/np.sum(w))-w_RV**2) * (x/(x-1))
            RV_f.append(np.average(g["RV"], weights=w))
            sRV_f.append(np.sqrt(var/x))
            eRV_f.append(np.average(g["eRV"]))
            RV_T.append(4)
    if x == 1:
        RV_f.append(g["RV"][0])
        sRV_f.append(0.0)
        eRV_f.append(g["eRV"][0])
        RV_T.append(1)

Finally, print all these results to a table.

In [ ]:
f_out = [f_un["name"], RV_f, sRV_f, eRV_f, RV_T]

f_table = Table(data=f_out, masked=False, names=('Name','RVfin','sRV','eRV','RV_T'))

f_table["Name"].format = '%s'
f_table["RVfin"].format = '%+9.2f'
f_table["sRV"].format = '%9.2f'
f_table["eRV"].format = '%9.2f'
f_table["RV_T"].format = '%i'

ascii.write(f_table, "finalRVs.csv", format='csv', overwrite=True)